In [138]:
import agents
import game
import mechanisms

In [139]:
import importlib
importlib.reload(agents)
importlib.reload(game)
importlib.reload(mechanisms)

<module 'mechanisms' from 'C:\\Users\\billz\\PycharmProjects\\CS_281\\Finding_Friends\\mechanisms.py'>

In [140]:
import numpy as np
import pandas as pd

import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

# if gpu is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [141]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    def push(self, *args):
        """Saves a transition."""
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = Transition(*args)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)
    
memory = ReplayMemory(10000)

In [142]:
class DQ_Learner(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(DQ_Learner, self).__init__()
        self.head = nn.Linear(input_dim, output_dim)
        
    def forward(self, x):
        return self.head(x)

class DQ_Agent(agents.Agent):
    def __init__(self, id, level, input_dim, output_dim):
        super(DQ_Agent, self).__init__(id, level)
        
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.learner = DQ_Learner(input_dim, output_dim).to(device)
        self.steps = 0
        
    def pick_friends(self, levels, cap, skill_levels=None):
        state = [cap] + levels
        if skill_levels is not None:
            state = state + skill_levels
        
        state = torch.tensor(state, device=device, dtype=torch.float)
        print(state)
        
        if self.last_reward is not None:
            memory.push(self.last_state, self.last_action, state, self.last_reward)
        
        # Perform one step of the optimization (on the target network)
        optimize_model()
        
        # Perform e-greedy policy 
        sample = random.random()
        eps_threshold = EPS_END + (EPS_START - EPS_END) * math.exp(-1. * self.steps / EPS_DECAY)
        self.steps += 1
        if sample > eps_threshold:
            print('Policy')
            with torch.no_grad():
#                 temp = self.learner(state)
#                 print(temp)
#                 print(temp.max(0))
#                 print(temp.max(0)[1])
#                 print(temp.max(0)[1].view(1, 1))
                action = self.learner(state).max(0)[1].view(1, 1)
        else:
            print('Explore')
            action = torch.tensor([[np.random.randint(1, len(levels))]], device=device, dtype=torch.long)
#         probs = self.learner(state)
#         candidates = list(zip([i for i in range(len(levels))], probs))
#         candidates = sorted(candidates, key=lambda t: t[1])
        
#         action = candidates[0][0]
        
        self.last_action = action
        self.last_state = state
        
        return action
    
    def accept_reward(self, reward, done):
        self.last_reward = torch.tensor([reward], device=device)
        if done:
            memory.push(self.last_state, self.last_action, None, self.last_reward)
            
            # Perform one step of the optimization (on the target network)
            optimize_model()
            

In [143]:
BATCH_SIZE = 128
GAMMA = 0.999
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 200
TARGET_UPDATE = 10

NUM_PLAYERS = 5
BASE_LVL = 0
LVL_CAP = 100
WIN_PROB = 0.4

ff_mech = mechanisms.Baseline_Mechanism(num_players=NUM_PLAYERS, p=WIN_PROB)

input_dim = ff_mech.input_dim()
output_dim = ff_mech.output_dim()

policy_agent = DQ_Agent(0, BASE_LVL, input_dim, output_dim)
target_agent = DQ_Agent(1000, BASE_LVL, input_dim, output_dim)

policy_net = policy_agent.learner
target_net = target_agent.learner
# policy_net = DQ_Learner(input_dim, output_dim).to(device)
# target_net = DQ_Learner(input_dim, output_dim).to(device)

target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

optimizer = optim.RMSprop(policy_net.parameters())

steps_done = 0

# Don't need this because using the agent instead
# def select_action(state):
#     global steps_done
#     sample = random.random()
#     eps_threshold = EPS_END + (EPS_START - EPS_END) * math.exp(-1. * steps_done / EPS_DECAY)
#     steps_done += 1
#     if sample > eps_threshold:
#         with torch.no_grad():
#             return policy_net(state).max(1)[1].view(1, 1)
#     else:
#         return torch.tensor([[random.randrange(n_actions)]], device=device, dtype=torch.long)

episode_scores = []

def plot_scores():
    plt.figure(2)
    plt.clf()
    scores_t = torch.tensor(episode_scores, dtype=torch.float)
    plt.title('Training...')
    plt.xlabel('Episode')
    plt.ylabel('Duration')
    plt.plot(scores_t.numpy())
    # Take 100 episode averages and plot them too
    if len(scores_t) >= 100:
        means = scores_t.unfold(0, 100, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(99), means))
        plt.plot(means.numpy())

    plt.pause(0.001)  # pause a bit so that plots are updated
    if is_ipython:
        display.clear_output(wait=True)
        display.display(plt.gcf())

In [144]:
policy_net.state_dict()

OrderedDict([('head.weight',
              tensor([[ 0.1522,  0.3403,  0.3369, -0.1684, -0.2320,  0.0282],
                      [ 0.2582,  0.1926,  0.1686, -0.0947, -0.0910, -0.0108],
                      [-0.1405,  0.3854, -0.2179, -0.3075, -0.3374,  0.1989],
                      [-0.0748,  0.0072,  0.0632, -0.0172, -0.0996, -0.1477],
                      [-0.1650,  0.0685, -0.2966,  0.3442,  0.3576, -0.3673]])),
             ('head.bias',
              tensor([-0.2603, -0.3429,  0.2978, -0.0718,  0.3076]))])

In [145]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    
    transitions = memory.sample(BATCH_SIZE)
    
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))
    
    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None, batch.next_state)), device=device, dtype=torch.uint8)
    non_final_next_states = torch.cat([s for s in batch.next_state if s is not None])
    
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)
    
    print(state_batch)
    
    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_action_values = policy_net(state_batch).gather(1, action_batch)
    
    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1)[0].
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0].detach()
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    loss = F.smooth_l1_loss(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    for param in policy_net.parameters():
        param.grad.data.clamp_(-1, 1)
    optimizer.step()

In [146]:
num_episodes = 50

ff_agents = [policy_agent] + [agents.Basic_Agent(i, BASE_LVL) for i in range(1, NUM_PLAYERS)]
ff_game = game.Game(players=ff_agents, mechanism=ff_mech, cap=LVL_CAP, logging_level='DEBUG')

for i_episode in range(num_episodes):
    ff_game.play()
    episode_scores.append(ff_game.levels[0])
    plot_scores()
    ff_game.reset()
    
    # Update the target network, copying all weights and biases in DQN
    if i_episode % TARGET_UPDATE == 0:
        target_net.load_state_dict(policy_net.state_dict())
        
print('Complete')
plt.ioff()
plt.show()

<Figure size 432x288 with 0 Axes>

tensor([100.,   0.,   0.,   1.,   0.,   1.])
Explore
tensor([100.,   1.,   1.,   2.,   0.,   2.])
Explore
tensor([100.,   3.,   2.,   3.,   0.,   2.])
Explore
tensor([100.,   4.,   4.,   3.,   1.,   2.])
Explore
tensor([100.,   7.,   5.,   4.,   3.,   3.])
Policy
tensor([100.,   8.,   6.,   4.,   3.,   3.])
Explore
tensor([100.,   9.,   8.,   4.,   4.,   3.])
Policy
tensor([100.,   9.,   9.,   5.,   5.,   4.])
Explore
tensor([100.,  11.,  11.,   6.,   7.,   5.])
Explore
tensor([100.,  12.,  11.,   6.,   8.,   7.])
Explore
tensor([100.,  15.,  13.,   7.,   9.,   8.])
Explore
tensor([100.,  17.,  14.,   7.,   9.,   9.])
Explore
tensor([100.,  18.,  16.,   7.,  10.,  11.])
Policy
tensor([100.,  19.,  17.,   7.,  13.,  12.])
Explore
tensor([100.,  20.,  17.,   7.,  14.,  12.])
Explore
tensor([100.,  20.,  17.,   7.,  15.,  13.])
Explore
tensor([100.,  20.,  17.,   7.,  15.,  13.])
Policy
tensor([100.,  21.,  19.,   8.,  17.,  15.])
Explore
tensor([100.,  21.,  20.,   8.,  17.,  16.])
Explo

RuntimeError: size mismatch, m1: [1 x 768], m2: [6 x 5] at c:\n\pytorch_1559129895673\work\aten\src\th\generic/THTensorMath.cpp:940

<Figure size 432x288 with 0 Axes>